In [1]:
import sys
sys.path[0]+='/..'
# from graph_dataset.dataset import GraphDataset
from rgcn_dataset import GraphDataset

from utilities import *
import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
from uc_class import UC

import gurobipy as gp
import json
from torch_geometric.data import HeteroData
import collections


In [2]:
import copy

class BipartiteGraphConvolution(torch_geometric.nn.MessagePassing):
    """
    The bipartite graph convolution is already provided by pytorch geometric and we merely need
    to provide the exact form of the messages being passed.
    """

    def __init__(self,emb_size):
        super().__init__("add")
            
        self.feature_module_left = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size)
        )
        self.feature_module_edge = torch.nn.Sequential(
            torch.nn.Linear(2, emb_size, bias=False)
        )
        self.feature_module_right = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size, bias=False)
        )
        self.feature_module_final = torch.nn.Sequential(
            torch.nn.LayerNorm(emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )
        
    

    def forward(self, left_features, edge_indices, edge_features, right_features):
        output = self.propagate(
            edge_indices,
            size=(left_features.shape[0], right_features.shape[0]),
            node_features=(left_features, right_features),
            edge_features=edge_features,
        )
        return output
            
    def message(self, node_features_i, node_features_j, edge_features):
        output = self.feature_module_final(
            self.feature_module_left(node_features_i)
            + self.feature_module_edge(edge_features)
            + self.feature_module_right(node_features_j)
        )
        return output
    


class GNNPolicy(torch.nn.Module):
    def __init__(self):
        super(GNNPolicy, self).__init__()
        emb_size=64
        self.obj_node = ['obj']
        # self.node_types=['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost','obj']
        self.relations = ['v2o','o2c','v2c','c2o','o2v','c2v']
        
        self.init_sizes=[7,7,7,4,4,4,4,4,4,4,1]
        edge_feats=2
        # self.num_relations=len(self.var_node)*len(self.con_node)
        self.vc_idx={}
        self.oc_idx={}
        self.ov_idx={}
        # self.node_types = ['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']    
        self.node_embedding = torch.nn.ModuleList()
        
        for i in [7,4,1]:
            lin = torch.nn.Sequential(
                torch.nn.LayerNorm(i),
                torch.nn.Linear(i, emb_size),
                torch.nn.ReLU(),
                torch.nn.Linear(emb_size,emb_size),
                torch.nn.ReLU()
            )
            self.node_embedding.append(lin)
        self.edge_embedding = torch.nn.LayerNorm(edge_feats)

        self.con1=torch.nn.Sequential()
        self.emb1=torch.nn.Sequential()
        self.con2=torch.nn.Sequential()
        self.emb2=torch.nn.Sequential()
        for rel in self.relations:
            self.con1.append(BipartiteGraphConvolution(emb_size))
            self.con2.append(BipartiteGraphConvolution(emb_size))
            self.emb1.append(self.down_scale(emb_size))
            self.emb2.append(self.down_scale(emb_size))


        self.ln = torch.nn.LayerNorm(emb_size)


        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, 1, bias=False)
        )
    def down_scale(self,emb_size):
        return torch.nn.Sequential(
            torch.nn.Linear(2 * emb_size, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )

    def trans_dimensions(self, g):
        self.con_node = ['Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost','state_variable']
        self.var_node = ['u','p','s','d','sc']
        
        data = copy.deepcopy(g)
        self.con_node = [c for c in self.con_node if 'x' in data[c]]
        self.var_node = [v for v in self.var_node if 'x' in data[v]]
        for v in self.var_node:
            if v == 'u':
                self.lps = data[v].x[:,-1].clone()
                for layer in self.node_embedding[0]:
                    # print()
                    data[v].x = layer(data[v].x)
                    if isinstance(layer, torch.nn.LayerNorm):
                        data[v].x[:,-1] = self.lps
                        
            else:
                data[v].x = self.node_embedding[0](data[v].x)

        
        for c in self.con_node:
            data[c].x = self.node_embedding[1](data[c].x)


        data['obj'].x = self.node_embedding[2](data['obj'].x)
        cnt=0
        for v in self.var_node:
            for c in self.con_node:
                if 'edge_attr' in data[v,'v2c',c]:
                    data[v,'v2c',c].edge_attr = self.edge_embedding(data[v,'v2c',c].edge_attr)
                    data[c,'c2v',v].edge_attr = data[v,'v2c',c].edge_attr
                    self.vc_idx[(v,c)]=cnt
                    self.vc_idx[(c,v)]=cnt
                else:
                    self.vc_idx[(v,c)]=-1
                    self.vc_idx[(c,v)]=-1
                cnt+=1
        cnt=0
        for c in self.con_node:
            if 'edge_attr' in data['obj','o2c',c]:
                data['obj','o2c',c].edge_attr = self.edge_embedding(data['obj','o2c',c].edge_attr)
                data[c,'c2o','obj'].edge_attr = data['obj','o2c',c].edge_attr
                self.oc_idx[('obj',c)]=cnt
                self.oc_idx[(c,'obj')]=cnt
            else:
                self.oc_idx[('obj',c)]=-1
                self.oc_idx[(c,'obj')]=-1
            cnt+=1
        cnt=0
        for v in self.var_node:
            if 'edge_attr' in data['obj','o2v',v]:
                data['obj','o2v',v].edge_attr = self.edge_embedding(data['obj','o2v',v].edge_attr)
                data[v,'v2o','obj'].edge_attr = data['obj','o2v',v].edge_attr
                self.ov_idx[('obj',v)]=cnt
                self.ov_idx[(v,'obj')]=cnt
            else:
                self.ov_idx[('obj',v)]=-1
                self.ov_idx[(v,'obj')]=-1
            cnt+=1

        self.target_nodes = [self.obj_node,self.con_node,self.con_node,self.obj_node,self.var_node,self.var_node]
        self.source_nodes = [self.var_node,self.obj_node,self.var_node,self.con_node,self.obj_node,self.con_node]
        return data
    def process_nodes(self, target_node, source_node, emb, conv, data, edge_type):
        if 'o' in edge_type and 'v' in edge_type:
            idx = self.ov_idx
        elif 'o' in edge_type and 'c' in edge_type:
            idx = self.oc_idx
        else:
            idx = self.vc_idx
        for node in target_node:
            x = torch.zeros_like(data[node].x)
            for other_node in source_node:
                if idx[(other_node, node)] != -1:
                    x += conv(
                        data[other_node].x, 
                        data[other_node, edge_type, node].edge_index, 
                        data[other_node, edge_type, node].edge_attr,
                        data[node].x
                    )
            data[node].x = emb(torch.cat([data[node].x, self.ln(x)], dim=-1))
    
    def forward(self, data):
        data = self.trans_dimensions(data)
        for i,(tar,sou,rel) in enumerate(zip(self.target_nodes,self.source_nodes,self.relations)):
            self.process_nodes(tar, sou, self.emb1[i], self.con1[i], data, rel)
        for i,(tar,sou,rel) in enumerate(zip(self.target_nodes,self.source_nodes,self.relations)):
            self.process_nodes(tar, sou, self.emb2[i], self.con2[i], data, rel)

        x = self.output_module(data['u'].x).sigmoid()
        return x


In [3]:
#set folder
uc_model = '3bin_1bin_ramp'
# instance = '60_c11_based_8_std'
instance = '80_c11_based_8_std'
# instance = '5_std'
model = 'tripartite_lpsolve'

# n=int(instance.split('_')[0])*24


model_save_path = f'../model_save/{uc_model}_{instance}_{model}/'
log_save_path =   f"../model_save/{uc_model}_{instance}_{model}_log/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'wb')

In [4]:
LEARNING_RATE = 1e-4
NB_EPOCHS =1000
BATCH_SIZE = 1
NUM_WORKERS = 1
ETA_MIN=1e-5

In [5]:
train_data = GraphDataset(f'../datasets/{instance}/{uc_model}/tripartite_lpsolve/train/')
valid_data = GraphDataset(f'../datasets/{instance}/{uc_model}/tripartite_lpsolve/valid/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [6]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best.pth'):
    state_dict = torch.load(model_save_path+'model_best.pth')
    PredictModel.load_state_dict(state_dict)
    print("载入模型成功")

载入模型成功


In [7]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    acc = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            if optimizer:
                weighted_sol = batch.weighted_sol
            else:
                weighted_sol = batch.opt_sol
            presol = predict(batch).squeeze(1)
            loss = loss_fn(presol, weighted_sol)
            preds = (presol > 0.5).float()  # Assuming threshold of 0.5
            acc += torch.sum(preds == batch.opt_sol).item()/len(batch.opt_sol)

            
            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed
    acc /= n_samples_processed
    return mean_loss,acc


In [8]:
def valid(predict, instance_set):
    # instance_set = ['80_c11_based_8_std','1080_c30_based_8_std']

    opt_loss=[]
    for ins in instance_set:

        uc_path = f'../UC_AF/{ins}.mod'
        uc = UC(uc_path)
        json_path = f'../datasets/{ins}/json/valid/'
        inf_cnt=0
        files = os.listdir(json_path)
        files = [file for file in files if file.endswith('.json')][::3]
        for file in files:
            with open(os.path.join(json_path, file),'r') as f:
                data = json.load(f)
            #参数
            pmax = uc.ThPimax
            Dt = data['Dt']
            Spin = data['Spin']
            u0 = data['u0']
            p0 = data['p0']
            on_off = data['on_off']
            #
            m3 = uc.get_3bin_model_1bin_ramp(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
            m1 = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)

            graph=get_Tripartite_graph_lp(m3).to(DEVICE)
            sol = predict(graph).detach().cpu().numpy().reshape(-1,24)
            sol = restore(sol,uc,data).reshape(-1).astype(int)
            obj=solve(m1,sol)
            opt_obj = data['obj']
            if obj == -1:
                inf_cnt+=1
            else:
                opt_loss.append((obj-opt_obj)/obj)
        if len(files)==0:
            print('no file')
    opt_loss=np.array(opt_loss)
    return opt_loss.mean()

In [9]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS/10, eta_min=ETA_MIN)
best_val_loss = 1
best_optloss=1
cur_epoch = 0
instance_set = ['80_c11_based_8_std']


In [10]:
for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss,train_acc = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.6f} Train acc:{train_acc:0.6f}",end='\t')
    valid_loss,valid_acc = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.6f} Valid acc:{valid_acc:0.6f}")
    optloss=valid(PredictModel,instance_set)
    print("optloss",optloss)
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print("best_loss",best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    if cur_epoch%20==0:
        torch.save(PredictModel.state_dict(),model_save_path+f'model_{cur_epoch}.pth')
    if optloss<best_optloss:
        best_optloss=optloss
        print("best_optloss",best_optloss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best_acc.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    #100个epochloss不下降就停止
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}  Valid acc:{valid_acc} optloss:{optloss} TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.113122 Train acc:0.963811	Epoch 0 Valid loss: 0.105308 Valid acc:0.965564
optloss 0.005700978271982452
best_loss 0.10530809862261542
best_optloss 0.005700978271982452
Epoch 1 Train loss: 0.112548 Train acc:0.963867	Epoch 1 Valid loss: 0.105562 Valid acc:0.965370
